In [47]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import json


In [48]:
f = open('translated_chatgpt.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
df_ai = pd.DataFrame(data)

In [49]:
import re

with open("trnews-64/trnews-64.train.raw") as fi:
    articles = re.split("\n\n", fi.read()) 

In [50]:
articles = articles[0:1000]

In [51]:
truncated = []
for article in articles:
    splitted = article.split(" ")
    truncated.append(splitted[0:200])


In [52]:
truncated_text_human = []
for trunc in truncated:
    truncated_text_human.append(" ".join(trunc))

In [53]:
#TRUNCATED HUMAN 44 ve AI 999 KARŞILIĞI YOK

In [54]:
df_human = pd.DataFrame(truncated_text_human, columns = ['Text'])

In [55]:
df_human = df_human.drop(44)

In [58]:
df_ai.iloc[44]['Text']

'Duruşmanın ilk duruşmalarında, Roj TV ile ilgili bir kararın Kasım ayında yapılması tahmin edildi, ancak tanık sayısı arttı ve belgelendirme zamanı geldiğinde, kararın Aralık ayında ilan edilmesi bekleniyor. Soruşturma ve deneme Kopenhag Şehir Mahkemesi, Danimarka\'da devam etmektedir. Son duruşmada, soruşturmaya liderlik eden Bölgesel Şef Savcı Veers Riisager tarafından hazırlanan belgeler, mahkeme için bir televizyon ekranında gözden geçirildi. Belgeler, Belçika\'daki ROJ NV stüdyolarında polis baskınları sırasında ele alındı, Roj TV için programları üretti. Belgeler, Roj TV\'nin yayın politikası hakkında terör örgütü önerileri içeriyordu ve bu da organizasyonun lideri Abdullah Öcalan ve yayınlarda 30 yıllık çatışmayı içeriyordu. 25 sayfalık belge " Yayın Politikaları ile ilgili kararsızlar" olarak adlandırıldı. Önceki duruşmalarda savcı, Türkiye\'deki askeri, polis ve sivillerin yaptığı saldırıları vurguladı. Ayrıca Eylül ve Ekim aylarında organize edilen terör saldırıları hakkında

In [59]:
df_human['label'] = 0

In [60]:
df_ai['label'] = 1

In [61]:
data = pd.concat([df_ai, df_human])


In [62]:
X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['label'], test_size=0.33, random_state=42)

In [67]:
y_val = y_test[0:330]
y_test = y_test[330:]
X_val = X_test[0:330]
X_test = X_test[330:]

In [68]:
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased')
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True)

train_labels = y_train.tolist()
val_labels = y_val.tolist()


In [69]:
train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(train_encodings['input_ids']),
        torch.tensor(train_encodings['attention_mask']),
        torch.tensor(train_labels))
val_dataset = torch.utils.data.TensorDataset(
        torch.tensor(val_encodings['input_ids']),
        torch.tensor(val_encodings['attention_mask']),
        torch.tensor(val_labels))

In [70]:
model = AutoModelForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-128k-cased', num_labels=2)
training_args= TrainingArguments(
    output_dir=f'./results_bert-base-turkish-128k-cased/fold/english',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=f'./logs_bert-base-turkish-128k-cased/fold/english',
    logging_steps=100)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda data: {'input_ids': torch.stack([item[0] for item in data]),
                                'attention_mask': torch.stack([item[1] for item in data]),
                                'labels': torch.stack([item[2] for item in data])})
trainer.train()

Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification 

Step,Training Loss
100,0.655000
200,0.391900




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=252, training_loss=0.4562872477940151, metrics={'train_runtime': 4256.9585, 'train_samples_per_second': 0.944, 'train_steps_per_second': 0.059, 'total_flos': 1057706442547200.0, 'train_loss': 0.4562872477940151, 'epoch': 3.0})

In [71]:
import torch
torch.save(model, 'model_1.sav')

In [72]:
# Make predictions on the test data
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)
num_test_samples = len(test_encodings['input_ids'])
test_labels = [0] * num_test_samples

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels))

predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)





***** Running Prediction *****
  Num examples = 331
  Batch size = 64


In [73]:
from sklearn.metrics import classification_report
print("Classification report:")
print(classification_report(y_test, preds))



Classification report:
              precision    recall  f1-score   support

           0       0.98      0.86      0.92       162
           1       0.88      0.98      0.93       169

    accuracy                           0.92       331
   macro avg       0.93      0.92      0.92       331
weighted avg       0.93      0.92      0.92       331

